<a href="https://colab.research.google.com/github/maecyntha/ai-classical-music-detector/blob/main/notebooks/02_lstm_ai_music_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras_tuner

## Data Pre-Processing

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load dataset
data = pd.read_csv("repeated.csv")
X = data.drop(columns=["is_ai"])
y = data["is_ai"]

# Split data
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=1/3, random_state=42)

print(f"Train: {X_train.shape}, Validation: {X_val.shape}, Test: {X_test.shape}")

In [ ]:
from sklearn.preprocessing import StandardScaler
import numpy as np

# Avoid warnings
non_constant_features = np.std(X_train, axis=0) > 0
non_constant_features = non_constant_features.to_numpy()

# Check if X_train is Pandas DataFrame
if isinstance(X_train, pd.DataFrame):
    X_train_selected = X_train.iloc[:, non_constant_features]
    X_val_selected = X_val.iloc[:, non_constant_features]
    X_test_selected = X_test.iloc[:, non_constant_features]
else:
    X_train_selected = X_train[:, non_constant_features]
    X_val_selected = X_val[:, non_constant_features]
    X_test_selected = X_test[:, non_constant_features]

# Only Normalize on feature with variation
scaler = StandardScaler()

X_train_normalized = scaler.fit_transform(X_train_selected)
X_val_normalized = scaler.transform(X_val_selected)
X_test_normalized = scaler.transform(X_test_selected)

num_features_per_segment = 9  # Pitch (3) + Velocity (3) + Duration (3)
num_segments = X_train_normalized.shape[1] // num_features_per_segment

# Reshape to 3D shape
X_train_normalized = X_train_normalized.reshape(X_train_normalized.shape[0], num_segments, num_features_per_segment)
X_val_normalized = X_val_normalized.reshape(X_val_normalized.shape[0], num_segments, num_features_per_segment)
X_test_normalized = X_test_normalized.reshape(X_test_normalized.shape[0], num_segments, num_features_per_segment)

## Model Training

### History Logger

In [ ]:
from tensorflow.keras.callbacks import Callback

all_histories = {}
trial_idx = -1

class SaveHistory(Callback):
    def on_train_begin(self, logs=None):
        global trial_idx
        trial_idx += 1

        all_histories[trial_idx] = {
            "loss": [], "val_loss": [], "accuracy": [], "val_accuracy": []
        }

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}

        all_histories[trial_idx]["loss"].append(logs.get("loss", 0))
        all_histories[trial_idx]["val_loss"].append(logs.get("val_loss", 0))
        all_histories[trial_idx]["accuracy"].append(logs.get("accuracy", 0))
        all_histories[trial_idx]["val_accuracy"].append(logs.get("val_accuracy", 0))

### Define Model

In [ ]:
import tensorflow as tf
import keras_tuner as kt
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional, Input, Masking
from tensorflow.keras.regularizers import l2

class LSTMHyperModel(kt.HyperModel):
  def build(self, hp):
    model = Sequential([
      Input(shape=(X_train_normalized.shape[1], X_train_normalized.shape[2])),
      LSTM(
          units=hp.Choice('lstm_units', [16, 32, 64]),
          kernel_regularizer=l2(hp.Float('lstm_regularizer', min_value=0.005, max_value=0.05, step=0.005))
      ),
      Dropout(hp.Float('dropout_rate', min_value=0.1, max_value=0.5, step=0.1)),
      Dense(
          1,
          activation="sigmoid",
          kernel_regularizer=l2(hp.Float('dense_regularizer', min_value=0.005, max_value=0.05, step=0.005))
      )
    ])

    model.compile(
      optimizer=hp.Choice('optimizer', ['adam', 'rmsprop']),
      loss="binary_crossentropy",
      metrics=["accuracy"]
    )

    return model

  def fit(self, hp, model, *args, **kwargs):
    return model.fit(
        *args,
        batch_size=hp.Choice("batch_size", [32, 64]),
        **kwargs,
    )

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Initiate Keras Tuner (RandomSearch)
tuner = kt.BayesianOptimization(
    LSTMHyperModel(),
    objective=[
        kt.Objective("val_loss", direction="min"),
        kt.Objective("val_accuracy", direction="max")
    ],
    max_trials=25,
    num_initial_points=25,
    overwrite=True,
    directory="keras_tuner_dir",
    project_name="lstm_tuning"
)

early_stopping = EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)
lr_reducer = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-5)
history_callback = SaveHistory()

# Find best hyperparameter
tuner.search(
    X_train_normalized, y_train,
    validation_data=(X_val_normalized, y_val),
    callbacks=[early_stopping, lr_reducer, history_callback],
    epochs=80,
    verbose=2
)

## Best Model Data & Plot

### Base Model

In [ ]:
best_trial = tuner.oracle.get_best_trials(num_trials=1)[0]

best_val_acc = best_trial.metrics.get_best_value('val_accuracy')
best_val_loss = best_trial.metrics.get_best_value('val_loss')

print("Best Trial (Hyperparameter Tuning)")
print(f"Accuracy: {best_val_acc:.4f}")
print(f"Loss: {best_val_loss:.4f}")

In [ ]:
# Get the best model
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.get_best_models(num_models=1)[0]

# Print best hyperparameter
print(f"LSTM Units: {best_hps.get('lstm_units')}")
print(f"LSTM Regularizer: {best_hps.get('lstm_regularizer')}")
print(f"Dropout Rate: {best_hps.get('dropout_rate')}")
print(f"Dense Regularizer: {best_hps.get('dense_regularizer')}")
print(f"Optimizer: {best_hps.get('optimizer')}")
print(f"Batch Size: {best_hps.get('batch_size')}")

In [ ]:
import matplotlib.pyplot as plt

best_trial_idx = best_trial.trial_id
best_trial_idx = int(best_trial_idx)
print(best_trial_idx)

best_history = all_histories.get(best_trial_idx, None)

plt.figure(figsize=(12, 5))

# Plot Loss
plt.subplot(1, 2, 1)
plt.plot(best_history["loss"], label="Training Loss")
plt.plot(best_history["val_loss"], label="Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Training & Validation Loss")
plt.legend()

# Plot Accuracy
plt.subplot(1, 2, 2)
plt.plot(best_history["accuracy"], label="Training Accuracy")
plt.plot(best_history["val_accuracy"], label="Validation Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("Training & Validation Accuracy")
plt.legend()

plt.show()


### Fine-Tuned Model

In [ ]:
import tensorflow as tf

best_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Callback to stop training if converged
early_stopping_finetune = EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)
lr_reducer = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-5)

# Fine-tune model using validation set
history_finetune = best_model.fit(
    X_train_normalized, y_train,
    validation_data=(X_val_normalized, y_val),
    epochs=30,
    batch_size=best_hps.get("batch_size"),
    callbacks=[early_stopping_finetune, lr_reducer],
    verbose=2
)


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 5))

# Plot Loss
plt.subplot(1, 2, 1)
plt.plot(history_finetune.history["loss"], label="Training Loss")
plt.plot(history_finetune.history["val_loss"], label="Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Training & Validation Loss (Fine-Tuning)")
plt.legend()

# Plot Accuracy
plt.subplot(1, 2, 2)
plt.plot(history_finetune.history["accuracy"], label="Training Accuracy")
plt.plot(history_finetune.history["val_accuracy"], label="Validation Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("Training & Validation Accuracy (Fine-Tuning)")
plt.legend()

plt.show()


## Test And Evaluation

### Primary Test Set

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import keras

y_pred = best_model.predict(X_test_normalized)
y_pred = (y_pred > 0.5).astype(int)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, digits=4)

print(f"Accuracy: {accuracy:.4f}")
print("Classification Report:\n", report)

matrix = confusion_matrix(y_test, y_pred)

# Transform to dataframe for easier plotting
matrix_df = pd.DataFrame(matrix,
                        index=['human', 'AI'],
                        columns=['human', 'AI'])

# Print Confusion Matrix
plt.figure(figsize=(3,3))
plt.title("Confusion Matrix")
sns.heatmap(matrix_df, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

In [ ]:
# Save trained LSTM model
best_model.save('lstm_best_model.keras')

### Auxiliary Test Set

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import keras

data = pd.read_csv("repeated_test.csv")
X_full_test = data.drop(columns=["is_ai"])
y_full_test = data["is_ai"]

# Fit dan transform pada data uji
X_full_test_normalized = scaler.transform(X_full_test)

num_features_per_segment = 9  # Pitch (3) + Velocity (3) + Duration (3)
num_segments = X_full_test.shape[1] // num_features_per_segment

# Reshape to 3D shape
X_full_test_normalized = X_full_test_normalized.reshape(X_full_test_normalized.shape[0], num_segments, num_features_per_segment)

y_pred = best_model.predict(X_full_test_normalized)
y_pred = (y_pred > 0.5).astype(int)

accuracy = accuracy_score(y_full_test, y_pred)
report = classification_report(y_full_test, y_pred, digits=4)

print(f"Accuracy: {accuracy:.4f}")
print("Classification Report:\n", report)

matrix = confusion_matrix(y_full_test, y_pred)

# Transform to dataframe for easier plotting
matrix_df = pd.DataFrame(matrix,
                        index=['human', 'AI'],
                        columns=['human', 'AI'])

# Print Confusion Matrix
plt.figure(figsize=(3,3))
plt.title("Confusion Matrix")
sns.heatmap(matrix_df, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()